In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType

In [4]:
import datetime as dt
from pyspark.sql import functions as F
from pyspark.sql import Window

# ==========================================
# 사용자 설정 (원하는 지역과 시군구를 입력하세요)
# ==========================================
REGION = "경기"
SIGUNGU_CODE = None  # 특정 시군구 처리 시 "41461" 입력, 전체 처리 시 None

# 앱 이름 및 경로용 시군구 문자열 처리
sigungu_dir = SIGUNGU_CODE if SIGUNGU_CODE else "all"

In [5]:
# Spark Session 설정
spark = SparkSession.builder \
    .appName(f'silver_stage_1_{REGION}_{sigungu_dir}') \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

# 날짜 계산 (직전 분기 말일 기준)
today = dt.date.today()
cur_q = (today.month - 1) // 3 + 1
prev_q = cur_q - 1
prev_q_year = today.year
if prev_q == 0:
    prev_q = 4
    prev_q_year -= 1
prev_q_last_month = prev_q * 3

# 베이스 경로
BUILDING_BASE = "/opt/spark/data/buildingLeader/parquet"
TOJI_BASE     = "/opt/spark/data/tojiSoyuJeongbo/parquet"
REST_BASE     = "/opt/spark/data/restaurant/parquet"

# 데이터 로드 경로
building_path = f"{BUILDING_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
toji_path     = f"{TOJI_BASE}/year={prev_q_year}/month={prev_q_last_month:02d}"
rest_path     = f"{REST_BASE}/dataset={prev_q_year}Q{prev_q}"

Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Picked up JAVA_TOOL_OPTIONS: -Dfile.encoding=UTF-8
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/13 02:55:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# 1) Building Load & Filter
building_df = spark.read.option("mergeSchema", "false").parquet(building_path)
building_gg_df = building_df.filter(F.col("region") == REGION)
if SIGUNGU_CODE:
    building_gg_df = building_gg_df.filter(F.col("시군구_코드") == SIGUNGU_CODE)

# 2) Toji Load & Filter
toji_df = spark.read.option("mergeSchema", "false").parquet(toji_path)
toji_gg_df = toji_df.filter(F.col("region") == REGION)
if SIGUNGU_CODE:
    toji_gg_df = toji_gg_df.filter(F.col("법정동코드").startswith(SIGUNGU_CODE))

# 3) Restaurant Load & Filter
restaurant_df = spark.read.option("mergeSchema", "false").parquet(rest_path)
restaurant_gg_df = restaurant_df.filter(F.col("region") == REGION)
if SIGUNGU_CODE:
    restaurant_gg_df = restaurant_gg_df.filter(F.col("법정동코드").startswith(SIGUNGU_CODE))

**Building Pruning**

In [7]:
building_gg_df = building_gg_df.withColumn(
    "고유번호",
    F.concat(
        F.col("시군구_코드"),        # 5
        F.col("법정동_코드"),        # 5
        F.when(F.col("대지_구분_코드") == "0", F.lit("1"))  # 대지 -> 1
         .when(F.col("대지_구분_코드") == "1", F.lit("2"))  # 산   -> 2
         .otherwise(F.col("대지_구분_코드")),
        F.lpad(F.col("번"), 4, "0"), # 본번
        F.lpad(F.col("지"), 4, "0")  # 부번
    )
)

building_gg_df = (
    building_gg_df
    .filter(F.col("대장_구분_코드") == "1")
    .select(
        F.col("관리_건축물대장_PK"),
        F.col("고유번호"),
        F.col("옥외_자주식_면적(㎡)").alias("옥외자주식면적")
    )
)

building_gg_df.printSchema()
print(building_gg_df.count())
building_gg_df.show(5, truncate=False)

root
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 고유번호: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)



1140412
+------------------+-------------------+--------------+
|관리_건축물대장_PK|고유번호           |옥외자주식면적|
+------------------+-------------------+--------------+
|109918059         |4122025930103930014|0             |
|1099111239        |4122010300104390000|0             |
|10991100508032    |4122037029100160002|27.5          |
|1099110636        |4122010100201370000|0             |
|10991100261948    |4122010200104920051|0             |
+------------------+-------------------+--------------+
only showing top 5 rows


**Toji Pruning**

In [8]:
toji_gg_df = (
    toji_gg_df
    # 1) 조건 필터
    .filter(F.col("공유인수") == 0)
    .filter(F.col("소유구분코드") == "01")
    # 2) 필요한 컬럼만 선택
    .select(
        F.col("고유번호").cast("string"),
        F.col("법정동명"),
        F.col("지번"),
        F.col("소유권변동원인"),
        F.col("소유권변동일자"),
        F.col("토지면적"),
        F.col("지목"),
        F.col("공시지가")
    )

    # 3) 날짜 파싱
    .withColumn(
        "소유권변동일자_dt",
        F.to_date(F.col("소유권변동일자"), "yyyy-MM-dd")
    )

    # 4) 고유번호별 최신 1행
    .withColumn(
        "rn",
        F.row_number().over(
            Window.partitionBy("고유번호")
                  .orderBy(F.col("소유권변동일자_dt").desc_nulls_last())
        )
    )
    .filter(F.col("rn") == 1)
    .drop("rn", "소유권변동일자_dt")

    # 5) 지번 분리
    .withColumn(
        "본번",
        F.split(F.col("지번"), "-").getItem(0)
    )
    .withColumn(
        "부번",
        F.when(
            F.size(F.split(F.col("지번"), "-")) > 1,
            F.split(F.col("지번"), "-").getItem(1)
        ).otherwise(F.lit(None))
    )
)

# 확인
toji_gg_df.printSchema()
print(toji_gg_df.count())
toji_gg_df.show(10, truncate=False)

root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동원인: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 공시지가: long (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)



2384743


[Stage 13:=============================>                            (2 + 2) / 4]

+-------------------+---------------------------+-----+--------------+--------------+--------+------+--------+----+----+
|고유번호           |법정동명                   |지번 |소유권변동원인|소유권변동일자|토지면적|지목  |공시지가|본번|부번|
+-------------------+---------------------------+-----+--------------+--------------+--------+------+--------+----+----+
|4111112900100020002|경기도 수원시 장안구 파장동|2-2  |소유권이전    |1993-05-04    |592.0   |임야  |8190    |2   |2   |
|4111112900100030008|경기도 수원시 장안구 파장동|3-8  |주소변경      |1983-09-26    |1342.0  |임야  |19400   |3   |8   |
|4111112900100110001|경기도 수원시 장안구 파장동|11-1 |소유권보존    |2010-04-16    |969.0   |답    |17100   |11  |1   |
|4111112900100190019|경기도 수원시 장안구 파장동|19-19|소유권이전    |2023-07-11    |47.0    |도로  |292500  |19  |19  |
|4111112900100220003|경기도 수원시 장안구 파장동|22-3 |주소변경      |2021-10-14    |939.0   |대    |710300  |22  |3   |
|4111112900100370002|경기도 수원시 장안구 파장동|37-2 |미등기        |NULL          |132.0   |도로  |0       |37  |2   |
|4111112900100390004|경기도 수원시 장안구 파장동|39-4 |소유권이전    |2025

**Restaurant Pruning**

In [9]:
restaurant_gg_df = (
    restaurant_gg_df
    .filter(F.col("상권업종대분류명") == "음식")
    .select(
    F.col("상호명"),
    F.col("지점명"),
    F.col("도로명"),
    F.col("상권업종대분류명"),
    F.col("상권업종중분류명"),
    F.col("상권업종소분류명"),
    F.col("지번코드"),
    F.col("경도").cast("double"),
    F.col("위도").cast("double")
    )
)
restaurant_gg_df.printSchema()
print(restaurant_gg_df.count())
restaurant_gg_df.show(5, truncate=False)

root
 |-- 상호명: string (nullable = true)
 |-- 지점명: string (nullable = true)
 |-- 도로명: string (nullable = true)
 |-- 상권업종대분류명: string (nullable = true)
 |-- 상권업종중분류명: string (nullable = true)
 |-- 상권업종소분류명: string (nullable = true)
 |-- 지번코드: string (nullable = true)
 |-- 경도: double (nullable = true)
 |-- 위도: double (nullable = true)

176901
+------------------+------+-------------------------------------+----------------+----------------+----------------------+-------------------+----------------+----------------+
|상호명            |지점명|도로명                               |상권업종대분류명|상권업종중분류명|상권업종소분류명      |지번코드           |경도            |위도            |
+------------------+------+-------------------------------------+----------------+----------------+----------------------+-------------------+----------------+----------------+
|세렌              |NULL  |경기도 성남시 분당구 운중로188번길   |음식            |서양식          |경양식                |4113511500110310004|127.083699272638|37.3898923202984|
|움터            

# 1

## 1-a

In [10]:
t = toji_gg_df.alias("t")
b = building_gg_df.alias("b")

toji_building_gg_df = (
    t.join(b, F.col("t.고유번호") == F.col("b.고유번호"), "left")
     .drop(F.col("b.고유번호"))
)

toji_building_gg_df.printSchema()
print(toji_building_gg_df.count())
toji_building_gg_df.show(5, truncate=False)

root
 |-- 고유번호: string (nullable = true)
 |-- 법정동명: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동원인: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 공시지가: long (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)



2514474


[Stage 29:=============>    (3 + 1) / 4][Stage 30:==========>       (3 + 2) / 5]

+-------------------+---------------------------+-----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|고유번호           |법정동명                   |지번 |소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|관리_건축물대장_PK|옥외자주식면적|
+-------------------+---------------------------+-----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|4111112900100020002|경기도 수원시 장안구 파장동|2-2  |소유권이전    |1993-05-04    |592.0   |임야|8190    |2   |2   |NULL              |NULL          |
|4111112900100030008|경기도 수원시 장안구 파장동|3-8  |주소변경      |1983-09-26    |1342.0  |임야|19400   |3   |8   |NULL              |NULL          |
|4111112900100110001|경기도 수원시 장안구 파장동|11-1 |소유권보존    |2010-04-16    |969.0   |답  |17100   |11  |1   |NULL              |NULL          |
|4111112900100190019|경기도 수원시 장안구 파장동|19-19|소유권이전    |2023-07-11    |47.0    |도로|292500  |19  |19  |NULL              |NULL          |
|4111112900100220003|경기도 수원시 장안구 파장동|22-3 |주소변경      |2

## 1-b

In [11]:
pk_cnt_df = (
    toji_building_gg_df
    .groupBy("고유번호")
    .agg(F.count("관리_건축물대장_PK").alias("pk_cnt"))
)

toji_binary_building_gg_df = (
    toji_building_gg_df
    .join(
        pk_cnt_df.filter(F.col("pk_cnt") <= 1),
        on="고유번호",
        how="inner"
    )
    .drop("pk_cnt")
)

print(toji_binary_building_gg_df.count())
toji_binary_building_gg_df.show(5, truncate=False)

2302729


[Stage 54:>                                                         (0 + 1) / 1]

+-------------------+---------------------------+-----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|고유번호           |법정동명                   |지번 |소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|관리_건축물대장_PK|옥외자주식면적|
+-------------------+---------------------------+-----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|4111112900100020002|경기도 수원시 장안구 파장동|2-2  |소유권이전    |1993-05-04    |592.0   |임야|8190    |2   |2   |NULL              |NULL          |
|4111112900100030008|경기도 수원시 장안구 파장동|3-8  |주소변경      |1983-09-26    |1342.0  |임야|19400   |3   |8   |NULL              |NULL          |
|4111112900100110001|경기도 수원시 장안구 파장동|11-1 |소유권보존    |2010-04-16    |969.0   |답  |17100   |11  |1   |NULL              |NULL          |
|4111112900100190019|경기도 수원시 장안구 파장동|19-19|소유권이전    |2023-07-11    |47.0    |도로|292500  |19  |19  |NULL              |NULL          |
|4111112900100220003|경기도 수원시 장안구 파장동|22-3 |주소변경      |2

In [12]:
toji_binary_building_gg_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), F.lit(0))
     .otherwise(F.lit(1))
).groupBy("건물개수").count().orderBy("건물개수").show()

[Stage 59:>                                                         (0 + 4) / 4]

+--------+-------+
|건물개수|  count|
+--------+-------+
|       0|1819739|
|       1| 482990|
+--------+-------+



# 2

## 2-a

In [13]:
toji_with_1_building_gg_df = (
    toji_binary_building_gg_df
    .filter(F.col("관리_건축물대장_PK").isNotNull())
)

# 확인
print(toji_with_1_building_gg_df.count())
toji_with_1_building_gg_df.show(5, truncate=False)

482990


[Stage 84:>                                                         (0 + 1) / 1]

+-------------------+---------------------------+----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|고유번호           |법정동명                   |지번|소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|관리_건축물대장_PK|옥외자주식면적|
+-------------------+---------------------------+----+--------------+--------------+--------+----+--------+----+----+------------------+--------------+
|4111112900100220003|경기도 수원시 장안구 파장동|22-3|주소변경      |2021-10-14    |939.0   |대  |710300  |22  |3   |10841100481831    |13            |
|4111112900100800002|경기도 수원시 장안구 파장동|80-2|주소변경      |2017-01-06    |255.0   |대  |965500  |80  |2   |10841100349920    |30            |
|4111112900100910001|경기도 수원시 장안구 파장동|91-1|소유권이전    |2007-04-13    |354.0   |대  |1004000 |91  |1   |10841100173407    |23            |
|4111112900100920004|경기도 수원시 장안구 파장동|92-4|주소변경      |2013-08-26    |205.0   |대  |965500  |92  |4   |1084114714        |23            |
|4111112900100930000|경기도 수원시 장안구 파장동|93  |소유권이전    |1997-

## 2-b

In [14]:
# 건물 있는데 음식점 아닌거 거르려고 만드는 join table

t = toji_with_1_building_gg_df.alias("t")
r = restaurant_gg_df.alias("r")

toji_building_restaurant_gg_df = (
    t.join(
        r,
        F.col("t.고유번호") == F.col("r.지번코드"),
        how="left"
    )
    .drop(F.col("r.지번코드"))
)

In [15]:
toji_building_restaurant_gg_df.select(
    F.count("*").alias("전체"),
    F.sum(F.col("상호명").isNull().cast("int")).alias("상호명_NULL"),
    F.sum(F.col("상호명").isNotNull().cast("int")).alias("상호명_NOT_NULL")
).show()

[Stage 93:>                                                         (0 + 4) / 4]

+------+-----------+---------------+
|  전체|상호명_NULL|상호명_NOT_NULL|
+------+-----------+---------------+
|498741|     436000|          62741|
+------+-----------+---------------+



In [16]:
toji_building_restaurant_gg_df = (
    toji_building_restaurant_gg_df
    .filter(F.col("상호명").isNotNull())
)

# 확인
print(toji_building_restaurant_gg_df.count())
toji_building_restaurant_gg_df.show(5, truncate=False)

62741


[Stage 125:>                                                        (0 + 1) / 1]

+-------------------+---------------------------+------+--------------+--------------+--------+----+--------+----+----+----------------------+--------------+----------------------------+------+-------------------------------------+----------------+----------------+----------------+----------------+----------------+
|고유번호           |법정동명                   |지번  |소유권변동원인|소유권변동일자|토지면적|지목|공시지가|본번|부번|관리_건축물대장_PK    |옥외자주식면적|상호명                      |지점명|도로명                               |상권업종대분류명|상권업종중분류명|상권업종소분류명|경도            |위도            |
+-------------------+---------------------------+------+--------------+--------------+--------+----+--------+----+----+----------------------+--------------+----------------------------+------+-------------------------------------+----------------+----------------+----------------+----------------+----------------+
|4111112900101070001|경기도 수원시 장안구 파장동|107-1 |주소변경      |2012-12-12    |185.0   |대  |879900  |107 |1   |108416667             |0            

## 2-c

In [17]:
# 아무 건물도 없는 필지
toji_with_0_building_gg_df = (
    toji_binary_building_gg_df
    .filter(F.col("관리_건축물대장_PK").isNull())
)

# 위에서 만든 join table과 concat을 위해 column 추가
toji_with_0_building_gg_df = (
    toji_with_0_building_gg_df
    .withColumn("상호명", F.lit(None).cast("string"))
    .withColumn("지점명", F.lit(None).cast("string"))
    .withColumn("도로명", F.lit(None).cast("string"))
    .withColumn("상권업종대분류명", F.lit(None).cast("string"))
    .withColumn("상권업종중분류명", F.lit(None).cast("string"))
    .withColumn("상권업종소분류명", F.lit(None).cast("string"))
    .withColumn("경도", F.lit(None).cast("double"))
    .withColumn("위도", F.lit(None).cast("double"))
)

final_toji_df = (
    toji_building_restaurant_gg_df
    .unionByName(toji_with_0_building_gg_df)
)

In [18]:
# 전체 row 수 = (건물1+음식점) + (건물0)
print("final:", final_toji_df.count())
print("건물1+음식점:", toji_building_restaurant_gg_df.count())
print("건물0:", toji_with_0_building_gg_df.count())

# 건물 개수 분포
final_toji_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), 0).otherwise(1)
).groupBy("건물개수").count().orderBy("건물개수").show()


final: 1882480


건물1+음식점: 62741


건물0: 1819739


[Stage 174:======================================>                  (4 + 2) / 6]

+--------+-------+
|건물개수|  count|
+--------+-------+
|       0|1819739|
|       1|  62741|
+--------+-------+



## 2-d

In [19]:
all_null_group_df = (
    final_toji_df
    .groupBy("법정동명", "본번")
    .agg(
        F.sum(F.col("상호명").isNotNull().cast("int")).alias("non_null_cnt")
    )
    .filter(F.col("non_null_cnt") == 0)
)

# 개수
all_null_group_df.count()

702197

In [20]:
clean_final_toji_df = (
    final_toji_df
    .join(
        all_null_group_df.select("법정동명", "본번"),
        on=["법정동명", "본번"],
        how="left_anti"
    )
)

# 결과 확인
clean_final_toji_df.count()

181528

In [21]:
# 건물 개수 분포
clean_final_toji_df.withColumn(
    "건물개수",
    F.when(F.col("관리_건축물대장_PK").isNull(), 0).otherwise(1)
).groupBy("건물개수").count().orderBy("건물개수").show()

[Stage 299:============================>                            (2 + 2) / 4]

+--------+------+
|건물개수| count|
+--------+------+
|       0|118787|
|       1| 62741|
+--------+------+



In [24]:
spark.stop()

# 3

## 3-a